In [ ]:
import sys
import time
import datetime
from train import *
from params import TrainingParams
import matplotlib.pyplot as plt
from IPython import display
%matplotlib inline

from params import TransferParams
from pipeline import *
import numpy as np

In [ ]:
params = TrainingParams()
# params.train_path='/home/ubuntu/work/data/unlabeled2017/*.jpg'
params.train_path='data/train/val2017/*.jpg'
params.style_path='data/starry_night.jpg'
params.batch_size = 1
params.num_epoch = 2
params.learn_rate = 0.001
params.total_train_sample = 128

In [ ]:
def report_loss(params, batch, epoch, total_cost):
    params.cost_history.append(total_cost)
    plt.plot(params.cost_history)
    display.display(plt.gcf())
    display.clear_output(wait=True)

In [ ]:
def report_loss_simple(params, batch, epoch, total_cost):
    time_remaining = 0
    if params.last_time is None:
        params.last_time = time.perf_counter()
    else:
        cur_time = time.perf_counter()
        elapsed = cur_time - params.last_time
        params.last_time = cur_time
        batches_per_epoch = (params.total_train_sample/params.batch_size)
        batch_remaining = batches_per_epoch*(params.num_epoch-epoch) - batch
        time_remaining = batch_remaining*elapsed
        
    sys.stdout.write("\r")
    sys.stdout.write("Batch %i of epoch %i. Total Loss: %s. ETA %s " % (batch, epoch, str(total_cost), str(datetime.timedelta(seconds=time_remaining))))
    sys.stdout.flush()

In [ ]:
params.cost_history = []
params.last_time = None
train(params, report_fn=report_loss, restore_epoch=None)

In [ ]:
def transfer(path, out_path=None):
    # init    
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    
    input_image = process_img(path).eval()
    input_shape = [1] + list(input_image.shape)
    
    # make the model
    input_placeholder = tf.placeholder(dtype=tf.float32, shape=input_shape, name='input_images')
    gen = SpriteGenerator(input_placeholder, 'SpriteGenerator')
    
    # restore
    params = TransferParams()
    saver = tf.train.Saver()
    saver.restore(sess, params.save_path)
    
    # generate and write out
    output = sess.run(gen.output, feed_dict={input_placeholder:np.stack([input_image])})
    if out_path is not None:
        output = tf.image.encode_jpeg(output[0])  
        write = tf.write_file(out_path, output)
        sess.run(write)
        print('Generate Done.')
    else:
        plt.imshow(output[0]/255.)    

In [ ]:
# transfer('data/golden_gate.jpg')

In [ ]:
def do():
    tf.reset_default_graph()
    sess = tf.InteractiveSession()
    
    input_shape = [2,256,256,3]
    input_placeholder = tf.placeholder(dtype=tf.float32, shape=input_shape, name='input_images')
    input_style = np.zeros([2,256,256,3])
    params = TrainingParams()

    gen = SpriteGenerator(input_placeholder, 'SpriteGenerator')
    vggTrain = VGG16(gen.output, 'train_vgg')
    vggRef = VGG16(input_placeholder, 'train_ref')

    J = total_loss(sess, input_placeholder, gen, vggTrain, vggRef, input_style, params)
    
    for t in tf.trainable_variables():
        print(t)

In [ ]:
# do()